In [6]:
pip install arabic-reshaper python-bidi


     ------------------------------------ 160.6/160.6 kB 957.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from PIL import ImageFont, ImageDraw, Image
import cv2
import numpy as np
import tensorflow as tf
import time
import arabic_reshaper
from bidi.algorithm import get_display

# Load model
model = tf.keras.models.load_model("mobilenet_finetuned.h5")

# Labels (your classes)
labels = ['ع',"ال","ا","ب","ض","د","ف","غ","ح","ه","ج","ك","خ","لا","ل","م","ن","ق","ر","ص","س","ش","ط","ت","ة","ذ","ث","و","ي","ظ","ز"] 
IMG_SIZE = 224

# Load font (make sure arial.ttf supports Arabic or replace with another Arabic font)
fontpath = "arial.ttf"
font = ImageFont.truetype(fontpath, 48)

cap = cv2.VideoCapture(0)

sentence = ""       # our collected text
last_added = time.time()
cooldown = 10      # seconds between adding letters

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        break

    # Preprocess
    img = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    # Predict
    pred = model.predict(img, verbose=0)
    class_id = np.argmax(pred)
    confidence = np.max(pred)

    if confidence > 0.95:
        class_name = labels[class_id]
    else:
        class_name = "?"

    # Append letter if confident & cooldown passed
    if class_name != "?" and time.time() - last_added > cooldown:
        sentence += class_name
        last_added = time.time()

    # Convert frame to PIL for drawing Arabic text
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(frame_pil)

    # Show current letter
    reshaped_letter = arabic_reshaper.reshape(class_name)
    bidi_letter = get_display(reshaped_letter)
    draw.text((30, 50), f"{bidi_letter} ({confidence:.2f})", font=font, fill=(0,255,0,0))

    # Show built sentence (reshaped + RTL)
    reshaped_sentence = arabic_reshaper.reshape(sentence)
    bidi_sentence = get_display(reshaped_sentence)
    draw.text((30, 120), bidi_sentence, font=font, fill=(255,0,0,0))

    
    frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
    cv2.imshow("Arabic Sign", frame)

    
    if cv2.waitKey(1) & 0xFF == ord("c"):
        sentence = ""

    
    if cv2.waitKey(1)  == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
